In [21]:
# The purpose of this script is to train the best possible detector for bounding box classification problem
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, Flatten, Input
from tensorflow.keras import Model
#from PIL import image

class ClassCNN(tf.keras.Model):

    def __init__(self, input_shape, output_shape, **kwargs):
        super(ClassCNN, self).__init__()
        self._input_shape  = input_shape   # = (28, 28, 1)
        self._output_shape = output_shape  # = 10

        self.conv1      = Conv2D(32, kernel_size=(3, 3), activation=tf.nn.relu)
        self.conv2      = Conv2D(64, kernel_size=(3, 3), activation=tf.nn.relu)
        self.maxpool    = MaxPooling2D(pool_size=(2, 2))
        self.dropout1   = Dropout(0.25, name='dropout1')
        self.flatten    = Flatten()
        self.dense1     = Dense(128, activation=tf.nn.relu)
        self.dropout2   = Dropout(rate=0.5, name='dropout2')
        self.activation = Dense(self._output_shape, activation=tf.nn.softmax)

        self.conv1.build(     (None,) + input_shape )
        self.conv2.build(     (None,) + tuple(np.subtract(input_shape[:-1],2)) + (32,) )
        self.maxpool.build(   (None,) + tuple(np.subtract(input_shape[:-1],4)) + (64,) )
        self.dropout1.build( tuple(np.floor_divide(np.subtract(input_shape[:-1],4),2)) + (64,) )
        self.dropout2.build( 128 )
        self.build(           (None,) + input_shape)


    def call(self, x, training=False, **kwargs):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.maxpool(x)
        if training:  x = self.dropout1(x)
        x = self.flatten(x)
        x = self.dense1(x)
        if training:  x = self.dropout2(x)
        x = self.activation(x)
        return x

In [22]:
with tf.device('/device:cpu:0'):    
    model = ClassCNN(input_shape=(28, 28, 1), output_shape=10)
    model( tf.random.uniform(shape=(128,28,28,1)) )  # required to call .summary() before .fit()
    model.summary()

Model: "class_cnn_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           multiple                  320       
_________________________________________________________________
conv2d_17 (Conv2D)           multiple                  18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 multiple                  0         
_________________________________________________________________
dropout1 (Dropout)           multiple                  0         
_________________________________________________________________
flatten_5 (Flatten)          multiple                  0         
_________________________________________________________________
dense_10 (Dense)             multiple                  1179776   
_________________________________________________________________
dropout2 (Dropout)           multiple                  

In [29]:
class subclass(Model):
    def __init__(self):
        super(subclass, self).__init__()
        self.conv1      = Conv2D(32, kernel_size=(3, 3), activation=tf.nn.relu)
        self.flatten    = Flatten()
        self.dense1     = Dense(1)
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.dense1(x)

    def model(self):
        x = Input(shape=(24, 24, 3))
        return Model(inputs=[x], outputs=self.call(x))



with tf.device('/device:cpu:0'):  
    sub = subclass()
    sub.model().summary()

AttributeError: 'NoneType' object has no attribute 'op'

In [33]:
dataset_1 = tf.data.Dataset.range(10, 20)
dataset_2 = tf.data.Dataset.range(60, 70)

In [36]:
combined_dataset = dataset_1.concatenate(dataset_2).repeat().shuffle(100)

In [38]:
for x in combined_dataset.take(60):
    print(x)

tf.Tensor(14, shape=(), dtype=int64)
tf.Tensor(13, shape=(), dtype=int64)
tf.Tensor(66, shape=(), dtype=int64)
tf.Tensor(64, shape=(), dtype=int64)
tf.Tensor(64, shape=(), dtype=int64)
tf.Tensor(66, shape=(), dtype=int64)
tf.Tensor(18, shape=(), dtype=int64)
tf.Tensor(65, shape=(), dtype=int64)
tf.Tensor(68, shape=(), dtype=int64)
tf.Tensor(68, shape=(), dtype=int64)
tf.Tensor(63, shape=(), dtype=int64)
tf.Tensor(60, shape=(), dtype=int64)
tf.Tensor(15, shape=(), dtype=int64)
tf.Tensor(68, shape=(), dtype=int64)
tf.Tensor(19, shape=(), dtype=int64)
tf.Tensor(10, shape=(), dtype=int64)
tf.Tensor(18, shape=(), dtype=int64)
tf.Tensor(69, shape=(), dtype=int64)
tf.Tensor(66, shape=(), dtype=int64)
tf.Tensor(69, shape=(), dtype=int64)
tf.Tensor(18, shape=(), dtype=int64)
tf.Tensor(15, shape=(), dtype=int64)
tf.Tensor(60, shape=(), dtype=int64)
tf.Tensor(14, shape=(), dtype=int64)
tf.Tensor(10, shape=(), dtype=int64)
tf.Tensor(10, shape=(), dtype=int64)
tf.Tensor(16, shape=(), dtype=int64)
t

In [52]:
with tf.device('/device:cpu:0'):
    t1 = tf.constant([1,2,3,4], dtype=tf.float32)
    t2 = tf.constant([1, 1, 0, 4], dtype=tf.float32)
    res = t1 == t2
    print(res)
    print(tf.reduce_mean(tf.cast(x=(t1==t2), dtype=tf.float32)))

tf.Tensor([ True False False  True], shape=(4,), dtype=bool)
tf.Tensor(0.5, shape=(), dtype=float32)


In [65]:
t1 = tf.constant([0.2,0.53,0.51,0.499])

tf.cast(t1 > 0.5, tf.float32)


<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0., 1., 1., 0.], dtype=float32)>